# Load classes and test Oanda Account

In [ ]:
# Import the necesary libraries
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import timedelta
import Oanda_Manager as om
import Signal_Gen as sg
import White_Strategy as ws
import Market_Simulator as MS
from Signal_Gen import Signal_Gen
from dateutil.relativedelta import relativedelta
from datetime import date
from datetime import timedelta
from pandas.tseries.offsets import BDay

In [ ]:
# Create an Oanda_Manager object to connect to Oanda API
Account_details_file = 'Account_details.csv'
oandaAcc =  om.Oanda_Manager(Account_details_file)
print(oandaAcc.account_list())

# Use Oanda_Manage.py to retrieve the historical data

In [ ]:
# Retrieve the index data as a time-series dataframe usign an arbitrary date range 
#https://developer.oanda.com/rest-live-v20/instrument-df/#CandlestickGranularity
asset_name="SPX500_USD"
granularity="H4"
#granularity="M"
from_date = pd.to_datetime('1-1-2003')
to_date =  pd.to_datetime(date.today())
candle_data = oandaAcc.get_all_candles_data(asset_name, granularity, from_date, to_date)
candle_data.tail()

In [ ]:
# Describe the data
index_stats = candle_data.describe()
index_stats = index_stats.transpose()
index_stats

In [ ]:
# Retrieve and Plot the medium closing price
SPX500_USD_H4 = candle_data['mid_c']
sp500_data = go.Scatter(x=SPX500_USD_H4.index, y=SPX500_USD_H4)
layout = go.Layout(title='Price Plot', xaxis=dict(title='Date'), yaxis=dict(title='price'))
fig = go.Figure(data=[sp500_data], layout=layout)
fig.show()

In [ ]:
#Store the cleanned data in the requested file
fileName= "SPX500_USD_H4.CSV"
oandaAcc.save_CSV_file(candle_data,SPX500_USD_H4,fileName)

# Generate the signals and store them in a CSV file

In [ ]:
#Create a Signal_Gen object with the stored file
fileName= "SPX500_USD_H4.CSV"
Signal_Generator = Signal_Gen(fileName)
SPX500_H4_signals=Signal_Generator.series
SPX500_H4_signals.head()

In [ ]:
# Calculate the moving averages 14 21 40
rolling_window=14
SPX500_H4_signals = Signal_Generator.Simple_moving_average(rolling_window, 'SMA_14')
rolling_window=21
SPX500_H4_signals = Signal_Generator.Simple_moving_average(rolling_window, 'SMA_21')
rolling_window=40
SPX500_H4_signals = Signal_Generator.Simple_moving_average(rolling_window, 'SMA_40')
# Calculate the slope with three months window 6*90 =540
rolling_window=540
SPX500_H4_signals = Signal_Generator.moving_slope(rolling_window, 'Slope_540')

In [ ]:
# Plot the index and the generated signals

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)

fig.add_trace(go.Scatter(x=SPX500_H4_signals.index, y=SPX500_H4_signals.mid_c,
                    mode='lines', name='Close Price', line=dict(color='dimgrey', width=2)), row=1, col=1)
fig.add_trace(go.Scatter(x=SPX500_H4_signals.index, y=SPX500_H4_signals.SMA_14,
                    mode='lines', name='SMA_14', line=dict(color='crimson', width=1)), row=1, col=1)
fig.add_trace(go.Scatter(x=SPX500_H4_signals.index, y=SPX500_H4_signals.SMA_21,
                    mode='lines', name='SMA_21', line=dict(color='darkgreen', width=1)), row=1, col=1)
fig.add_trace(go.Scatter(x=SPX500_H4_signals.index, y=SPX500_H4_signals.SMA_40,
                    mode='lines', name='SMA_40', line=dict(color='cornflowerblue', width=1)), row=1, col=1)
fig.add_trace(go.Scatter(x=SPX500_H4_signals.index, y=SPX500_H4_signals.Slope_540,
                    mode='lines',name='Slope_540',line=dict(color='darkorange', width=2)), row=2, col=1)

fig.update_layout(autosize=False, width=1100,height=600)

fig.show()

In [ ]:
#Store the signals in the requested file
fileName= "SPX500_H4_SIGNALS.CSV"
Signal_Generator.save_CSV_file(SPX500_H4_signals,fileName)

# Execute the white robot strategy

In [ ]:
fileName= "SPX500_H4_SIGNALS.CSV"
White_Strat  = ws.White_Strategy(fileName)
#White_Strat.series.tail()

In [ ]:
order_book = White_Strat.simple_white_strat()

In [ ]:
#Store the signals in the requested file
fileName= "WR_ORDER_BOOK.CSV"
White_Strat.save_CSV_file(order_book,fileName)
oder_book.tail()

# Run Market Simulator - White Robot

In [ ]:
order_book = pd.read_csv(fileName)
Cash = 10000
MS = MS.Market_Sim(order_book, Cash)

In [ ]:
Portfolio_Simulation = MS.Portfolio_Simulation_csv_outputs()

## Visualising the Portfolio Simulation

In [ ]:
Portfolio_Simulation = pd.read_csv('Portfolio_Simulation.csv')
Portfolio_Simulation.head()

In [ ]:
new_fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02)

In [ ]:
new_fig.add_trace(go.Scatter(x=Portfolio_Simulation.time, y=Portfolio_Simulation.mid_c,
                    mode='lines', name='Close Price', line=dict(color='dimgrey', width=2)), row=1, col=1)
new_fig.add_trace(go.Scatter(x=Portfolio_Simulation.time, y =Portfolio_Simulation.Signals,
                     mode='lines', name='Signals', line=dict(color='blue', width=2)), row=2, col=1)
new_fig.add_trace(go.Scatter(x=Portfolio_Simulation.time, y =Portfolio_Simulation['Portfolio Value'],
                     mode='lines', name='Portfolio Value', line=dict(color='red', width=2)), row=3, col=1)
new_fig.update_layout(autosize=False, width=1100,height=600)